<a href="https://colab.research.google.com/github/vitor-marinho-fjp/biomas_mg/blob/main/biomas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# Carregar dados
df_mapbiomas = pd.read_excel("tabela_geral_mapbiomas_col8_biomas_municipios.xlsx", sheet_name="COBERTURA_COL8.0")

# Pivotar os dados
df_pivotado = df_mapbiomas.melt(id_vars=["geocode"], var_name="ano", value_name="valor")

# Calcular a área municipal por município
area_municipal_por_municipio = df_pivotado.groupby(["geocode", "ano"])["valor"].sum() / 100
area_municipal_por_municipio = area_municipal_por_municipio.reset_index(name="area_municipal")

# Filtrar as classes desejadas
classes_desejadas = [3, 4, 11, 12, 13, 29]
formacoes_desejadas = ['Forest Formation', 'Savanna Formation', 'Wetland', 'Grassland',
                       'Other Non Forest Natural Formation', 'Rocky outcrop']

dados = df_pivotado[(df_pivotado["state_acronym"] == 'MG') &
                    (df_pivotado["class_id"].isin(classes_desejadas)) &
                    (df_pivotado["level_4"].isin(formacoes_desejadas))]

# Calcular a área nativa
dados2 = dados.groupby(["ano", "geocode"])["valor"].sum() / 100
dados2 = dados2.reset_index(name="area_nativa")

# Juntar dados de área nativa com área municipal e calcular M_FLONATIVA
dados2 = pd.merge(dados2, area_municipal_por_municipio, on=["geocode", "ano"])
dados2["M_FLONATIVA"] = (dados2["area_nativa"] / dados2["area_municipal"]) * 100

# Salvar os dados
dados2.to_excel('area_cobertura.xlsx', index=False)
